# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-or-v1"):
    print("An API key was found, but it doesn't start sk-or-v1; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
# constants
MODEL_GPT = 'gpt-4o-mini'

open_router = "https://openrouter.ai/api/v1"
openai = OpenAI(base_url=open_router, api_key=api_key)

# set up environment
system_prompt="""
You are a clear and patient teacher.
Your job is to explain complex topics in a way a smart 12-year-old can understand.
Rules:
- Use simple, everyday language.
- Avoid jargon. If you must use a technical word, define it clearly.
- Use short paragraphs.
- Use at least one real-world analogy.
- Give 3 key takeaways at the end in bullet points.
- Do not assume prior knowledge.
- Be accurate but simple.
- Keep the explanation under 400 words.

Structure your response like this:

1) Simple Definition  
2) How It Works (Step-by-step if needed)  
3) Real-Life Example or Analogy  
4) 3 Key Takeaways
"""

In [ ]:
def get_user_prompt(chat, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}]  + history + [{"role": "user", "content": chat}]
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)
    stream = ""
    for chunk in response:
        stream += chunk.choices[0].delta.content or ''
        yield stream

In [ ]:
chat = gr.ChatInterface(fn=get_user_prompt, type="messages", title="AI Tutor", description="I help you understand complex topics with simple explanations!")

In [ ]:
chat.launch()